In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Dataset
from azureml.data.datapath import DataPath
from azureml.core import Environment


ws = Workspace.from_config()
print(f'Ready to use Azure ML {azureml.core.VERSION} to work with {ws.name}')

In [ ]:
default_ds = ws.get_default_datastore()
Dataset.File.upload_directory(src_dir='dataset9.01.2023', target=DataPath(default_ds, 'leaves/'))

In [ ]:
file_data_set = Dataset.File.from_files(path=(default_ds, 'leaves/*/*.jpg'))

try:
    file_data_set = file_data_set.register(workspace=ws, name='leaves dataset', create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

In [ ]:
experiment_env = Environment.get(ws, "AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu")
# experiment_env.python.user_managed_dependencies = False 

print(experiment_env.name, 'defined.')

In [29]:
dataset = Dataset.get_by_name(workspace=ws, name="leaves dataset", version=1)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "sw-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6', max_nodes=1, idle_seconds_before_scaledown=600)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

In [48]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice


script_config = ScriptRunConfig(source_directory="scripts",
                                script='train_script.py',
                                arguments = ['--input-data', dataset.as_named_input('training_files').as_download()], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment

params = GridParameterSampling(
    {
        '--initial_learning_rate': choice(0.001, 0.00001),
        '--optimizer' : choice("Adam", "RMSprop"),
        '--mobilenet_version': choice("Small", "Large"),
        '--augmentation_variant': choice(1, 2, 3, 4),
        '--limit_artificials': choice(0, 2500, 5000)
    }
)

hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None,
                          primary_metric_name='MAE', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_total_runs=96, 
                          max_concurrent_runs=8) 

experiment = Experiment(workspace=ws, name='sw_experiment_final')
run = experiment.submit(config=hyperdrive)